In [1]:
import pandas as pd
import numpy as np

In [50]:
def estimate_volatility(prices, l):
    """Create an exponential moving average model of the volatility of a stock
    price, and return the most recent (last) volatility estimate.
    
    Parameters
    ----------
    prices : pandas.Series
        A series of adjusted closing prices for a stock.
        
    l : float
        The 'lambda' parameter of the exponential moving average model. Making
        this value smaller will cause the model to weight older terms less 
        relative to more recent terms.
        
    Returns
    -------
    last_vol : float
        The last element of your exponential moving averge volatility model series.
    
    """
    # get log returns from stock prices and square them 
    log_returns_sq = (np.log(prices) - np.log(prices.shift(1))) ** 2
    
    # take the exponential moving weighted average of the returns using 
    # lambda parameter
    ret_emwa = log_returns_sq.ewm(alpha=1-l).mean()
    
    # return the square root of the final variance to return the standard deviation
    return ret_emwa.iloc[-1]**0.5    
    
    
def test_run(filename='data.csv'):
    """Test run get_most_volatile() with stock prices from a file."""
    prices = pd.read_csv(filename, parse_dates=['date'], index_col='date', squeeze=True)
    print("Most recent volatility estimate: {:.6f}".format(estimate_volatility(prices, 0.7)))


if __name__ == '__main__':
    test_run()

Most recent volatility estimate: 0.004941


### Visualising exponential moving weighted average

##### Adjusted False

$   y_t = \alpha x_t + (1 - \alpha) y_{t-1}$

In [119]:
alpha = 0.5
temperatures = [25, 28, 26]
series_temps = pd.Series(temperatures)
temps_ewm = series_temps.ewm(alpha=alpha,adjust=False).mean()

print(series_temps)

print('\nExpected Values')
print(25)
print(0.5 * 25 + 0.5 * 28)
print(26.5 * 0.5 + 26 * 0.5)

print('\nPanda values:')
print(temps_ewm)

0    25
1    28
2    26
dtype: int64

Expected Values
25
26.5
26.25

Panda values:
0    25.00
1    26.50
2    26.25
dtype: float64


##### Adjusted True

$$
y_t = \frac{x_t + (1 - \alpha)x_{t-1} + (1 - \alpha)^2 x_{t-2} + ...
+ (1 - \alpha)^t x_{0}}{1 + (1 - \alpha) + (1 - \alpha)^2 + ...
+ (1 - \alpha)^t}
$$

In [117]:
temps_ewm = series_temps.ewm(alpha=0.5,adjust=True).mean()

print('\nExpected Values')
print(25)
print((28+(1 - alpha)*25)/(1 + (1 - alpha)))
print((26 + (1-alpha)*28 + (1-alpha)**2*25)/(1 + (1-alpha) + (1-alpha)**2))

print('\nPanda values:')
print(temps_ewm)


Expected Values
25
27.0
26.428571428571427

Panda values:
0    25.000000
1    27.000000
2    26.428571
dtype: float64
